In [3]:
import numpy as np
import scipy as sp
import scipy.io
from scipy.stats import unitary_group
from datetime import datetime
import matplotlib.pyplot as plt
import qutip as qt
#from scipy.optimize import curve_fit
import os
import uuid
from itertools import repeat, chain, product, combinations
from joblib import Parallel, delayed
import glob
import time
from EMQST_lib import support_functions as sf
from EMQST_lib import emqst
from EMQST_lib.qst import QST  
from EMQST_lib.povm import POVM
from EMQST_lib import dt
from EMQST_lib import measurement_functions as mf
from EMQST_lib import visualization as vis
from EMQST_lib import overlapping_tomography as ot
from EMQST_lib import photon as ph
from functools import reduce
from cvxopt import matrix, solvers
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=2)

Comparing integrated coincidence counts to the background counts, counts on each beam line before before integration starts

In [9]:
is_math_file = True
if is_math_file:
    
    path  = "Photon_REM/photonic QREM experimental data v3/13 HVVH with lens/HVVH_lens.mat"
    compare_path_lens = "Photon_REM/photonic QREM experimental data v3-singles/11 HLVR with lens/HLVR_lens_singles.mat"
    compare_path_no_lens = "Photon_REM/photonic QREM experimental data v3-singles/02 HLVR no lens/HLVR_singles.mat"
    mat_files = glob.glob(path)[::-1] # To get order HVVH, HLVR, HDVA
    compare_files = glob.glob(compare_path_lens)[::-1] # To get order HVVH, HLVR, HDVA
    compare_files_no_lens = glob.glob(compare_path_no_lens)[::-1] # To get order HVVH, HLVR, HDVA
    print(mat_files)
    print(compare_files)
    print(compare_files_no_lens)
    #mat_files = [glob.glob(p + "/*.mat")[0] for p in paths]

    # Load from math files
    QST_dic = [sp.io.loadmat(f) for f in mat_files]
    #print(QST_dic[0]['HVVH_lens_coincRaw'])
    #print(QST_coinc_count_dic)
    QST_coinc_keys = [list(mat.keys()) for mat in QST_dic]
    print(QST_coinc_keys)
    # QST_calib_keys = [list(mat.keys())[3] for mat in QST_dic]
    # #print(QST_coinc_keys)
    # QST_coinc_counts = np.array([mat[key].T[0] for mat, key in zip(QST_dic, QST_coinc_keys)])
    
    compare_dic = [sp.io.loadmat(f) for f in compare_files]
    compare_path_no_lens = [sp.io.loadmat(f) for f in compare_files_no_lens]
    #print(compare_dic[0]["HVVH_lens_singles"])
    # compare_QST_coinc_keys = [list(mat.keys())[6] for mat in compare_dic]
    # compare_QST_calib_keys = [list(mat.keys())[3] for mat in compare_dic]
    # compare_QST_coinc_counts = np.array([mat[key].T[0] for mat, key in zip(compare_dic, compare_QST_coinc_keys)])

coincidene_order_QST = ['HH', 'HV', 'VV', 'VH',
                        'RH', 'RV', 'DV', 'DH',
                        'DR', 'DD', 'RD', 'HD',
                        'VD', 'VL', 'HL', 'RL']

for i in range(len(coincidene_order_QST)):
    print(coincidene_order_QST[i],compare_dic[0]["HLVR_lens_singles"][i], compare_path_no_lens[0]["HLVR_singles"][i])



['Photon_REM/photonic QREM experimental data v3/13 HVVH with lens/HVVH_lens.mat']
['Photon_REM/photonic QREM experimental data v3-singles/11 HLVR with lens/HLVR_lens_singles.mat']
['Photon_REM/photonic QREM experimental data v3-singles/02 HLVR no lens/HLVR_singles.mat']
[['__header__', '__version__', '__globals__', 'HVVH_lens', 'HVVH_lens_coinc', 'HVVH_lens_coincAcc', 'HVVH_lens_coincRaw']]
HH [11889  8373] [11521 21216]
HV [11938  8519] [11638 20958]
VV [9546 8391] [ 8996 20962]
VH [9373 8364] [ 9105 21358]
RH [10674  8440] [10226 21264]
RV [10671  8337] [10317 21031]
DV [10409  8494] [10126 21002]
DH [10447  8699] [10095 21640]
DR [10498  8123] [10120 20620]
DD [10392  8290] [10223 20939]
RD [10622  8474] [10492 20993]
HD [12047  8298] [11438 21126]
VD [9446 8378] [ 8984 20913]
VL [9369 8293] [ 9065 20976]
HL [11674  8493] [11493 21064]
RL [10596  8571] [10088 21260]
